In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,HuberRegressor, LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
weather_2009 = pd.read_excel('weather.xlsx',sheet_name = 0)
weather_2010 = pd.read_excel('weather.xlsx',sheet_name = 1)
weather_2011 = pd.read_excel('weather.xlsx',sheet_name = 2)
weather_2012 = pd.read_excel('weather.xlsx',sheet_name = 3)
weather_2013 = pd.read_excel('weather.xlsx',sheet_name = 4)
weather_2014 = pd.read_excel('weather.xlsx',sheet_name = 5)
weather_2015 = pd.read_excel('weather.xlsx',sheet_name = 6)
#weather_2016 = pd.read_excel('weather.xlsx',sheet_name = 7)
micro=pd.read_excel("macro_economic.xlsx")

In [4]:
train.shape, test.shape, weather_2009.shape, weather_2010.shape, weather_2011.shape, weather_2012.shape, weather_2013.shape, weather_2014.shape , weather_2015.shape, micro.shape #, weather_2016.shape

((216, 4),
 (36, 4),
 (365, 23),
 (365, 23),
 (365, 23),
 (366, 23),
 (365, 23),
 (365, 23),
 (365, 23),
 (96, 18))

In [5]:
#train.head(10)

In [6]:
train.isnull().sum()

Year                0
Month               0
ProductCategory     0
Sales              12
dtype: int64

In [7]:
month_cat_median=train.groupby(['Month','ProductCategory']).Sales.transform('median')
#month_cat_median

In [8]:
train.Sales.fillna(month_cat_median, inplace=True)
train.isnull().sum()

Year               0
Month              0
ProductCategory    0
Sales              0
dtype: int64

In [9]:
test.Sales.fillna(month_cat_median, inplace=True)
test.isnull().sum()

Year               0
Month              0
ProductCategory    0
Sales              0
dtype: int64

In [10]:
train_feature = train.drop(labels = "Sales", axis = 1)
test_feature = test.drop(labels = "Sales", axis = 1)

In [11]:
data1 = pd.concat((weather_2009, weather_2010), axis =0)
data2 = pd.concat((data1, weather_2011), axis =0)
data3 = pd.concat((data2, weather_2012), axis =0)
data4 = pd.concat((data3, weather_2013), axis =0)
data = pd.concat((data4, weather_2014), axis =0)
#X_Test = pd.concat((weather_2015, weather_2016), axis =0)

In [12]:
data.shape

(2191, 23)

In [13]:
data.isnull().sum()

Year                              0
Month                             0
Day                               0
Temp high (°C)                    0
Temp avg (°C)                     0
Temp low (°C)                     0
Dew Point high (°C)               0
Dew Point avg (°C)                0
Dew Point low (°C)                0
Humidity (%) high                 0
Humidity (%) avg                  0
Humidity (%) low                  0
Sea Level Press. (hPa) high       0
Sea Level Press. (hPa) avg        0
Sea Level Press. (hPa) low        0
Visibility (km) high              0
Visibility (km) avg               0
Visibility (km) low               0
Wind (km/h) low                   0
Wind (km/h) avg                   0
Wind (km/h) high                  0
Precip. (mm) sum                  0
WeatherEvent                   1412
dtype: int64

In [14]:
data.Month.replace(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',]
                   ,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)
weather_2015.Month.replace(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',]
                   ,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

In [15]:
normal = "Normal"
data.WeatherEvent.fillna(normal, inplace = True)
weather_2015.WeatherEvent.fillna(normal, inplace = True)

In [16]:
#labels=LabelEncoder()
data.WeatherEvent.replace(["Fog", "Fog , Rain", "Fog , Rain , Snow", "Fog , Snow", "Normal", "Rain", "Rain , Snow", "Snow", "Thunderstorm"   ]
                   ,[1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)

weather_2015.WeatherEvent.replace(["Fog", "Fog , Rain", "Fog , Rain , Snow", "Fog , Snow", "Normal", "Rain", "Rain , Snow", "Snow", "Thunderstorm"   ]
                   ,[1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)

In [17]:
train_feature.ProductCategory.replace(["MenClothing", "OtherClothing", "WomenClothing"],[1, 2, 3], inplace=True)
test_feature.ProductCategory.replace(["MenClothing", "OtherClothing", "WomenClothing"],[1, 2, 3], inplace=True)

In [18]:
#data.isnull().sum()

In [19]:
for i in list(range(22)):
    data.iloc[:,i]=pd.to_numeric(data.iloc[:,i], errors='coerce')

In [20]:
data.dtypes

Year                             int64
Month                            int64
Day                              int64
Temp high (°C)                 float64
Temp avg (°C)                  float64
Temp low (°C)                  float64
Dew Point high (°C)            float64
Dew Point avg (°C)             float64
Dew Point low (°C)             float64
Humidity (%) high              float64
Humidity (%) avg               float64
Humidity (%) low               float64
Sea Level Press. (hPa) high    float64
Sea Level Press. (hPa) avg     float64
Sea Level Press. (hPa) low     float64
Visibility (km) high           float64
Visibility (km) avg            float64
Visibility (km) low            float64
Wind (km/h) low                float64
Wind (km/h) avg                float64
Wind (km/h) high               float64
Precip. (mm) sum               float64
WeatherEvent                     int64
dtype: object

In [21]:
data = data.drop(labels = 'Day', axis = 1)

In [22]:
data_group=data.groupby(['Year','Month']).transform('median')

In [23]:
##assigning month and year feature to month median df
mt=data.iloc[:,1]
yr=data.iloc[:,0]
data_group = data_group.assign(Month=mt.values)
data_group = data_group.assign(Year=yr.values)
data_group.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,Year
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
1,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
2,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
3,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
4,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009


In [24]:
data_group.shape

(2191, 22)

In [25]:
drop_df=data_group.drop_duplicates(subset=None, keep='first', inplace=False)
drop_df.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,Year
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
31,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,16.0,16.0,16.0,26.5,12.0,40.0,0.0,5.0,2,2009
59,11.0,7.0,3.0,1.0,-1.0,-6.0,70.0,56.0,39.0,1026.0,...,16.0,16.0,14.0,27.0,12.0,39.0,0.0,5.0,3,2009
90,15.5,11.5,7.0,7.5,2.5,-2.0,72.0,54.5,31.0,1021.5,...,16.0,16.0,16.0,26.5,10.5,39.0,0.0,5.0,4,2009
120,21.0,17.0,12.0,14.0,11.0,7.0,93.0,69.0,46.0,1021.0,...,16.0,15.0,11.0,23.0,8.0,32.0,0.0,5.0,5,2009


In [26]:
drop_df1=drop_df.append(drop_df)
drop_df1.shape

(144, 22)

In [27]:
final_df=drop_df1.append(drop_df)
final_df.shape

(216, 22)

In [28]:
##sorting the final_df
final=final_df.sort_values(['Year','Month'], ascending=True)
final.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,Year
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009
31,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,16.0,16.0,16.0,26.5,12.0,40.0,0.0,5.0,2,2009
31,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,16.0,16.0,16.0,26.5,12.0,40.0,0.0,5.0,2,2009


In [29]:
final.shape

(216, 22)

In [30]:
cat = train_feature.iloc[:,2]
final= final.assign(ProductCategory = cat.values)

In [31]:
final.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,Year,ProductCategory
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009,3
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009,1
0,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,16.0,16.0,26.0,11.0,39.0,0.0,5.0,1,2009,2
31,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,16.0,16.0,26.5,12.0,40.0,0.0,5.0,2,2009,3
31,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,16.0,16.0,26.5,12.0,40.0,0.0,5.0,2,2009,1


In [32]:
final= final.drop(labels = "Year", axis =1)

In [33]:
final.shape

(216, 22)

In [34]:
weather_2015.isnull().sum()

Year                           0
Month                          0
Day                            0
Temp high (°C)                 0
Temp avg (°C)                  0
Temp low (°C)                  0
Dew Point high (°C)            0
Dew Point avg (°C)             0
Dew Point low (°C)             0
Humidity (%) high              0
Humidity (%) avg               0
Humidity (%) low               0
Sea Level Press. (hPa) high    0
Sea Level Press. (hPa) avg     0
Sea Level Press. (hPa) low     0
Visibility (km) high           0
Visibility (km) avg            0
Visibility (km) low            0
Wind (km/h) low                0
Wind (km/h) avg                0
Wind (km/h) high               0
Precip. (mm) sum               0
WeatherEvent                   0
dtype: int64

In [35]:
weather_2015.head()

,Year,Month,Day,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,Sea Level Press. (hPa) avg,Sea Level Press. (hPa) low,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent
0,2015,1,1,4,1,-3,-8,-12,-14,46,...,1018,1015,16,16,16,27,11,50,0,5
1,2015,1,2,6,4,2,-7,-8,-9,52,...,1023,1016,16,16,16,24,11,40,0,5
2,2015,1,3,6,3,1,4,-2,-9,92,...,1030,1019,16,7,1,19,10,29,18.03,7
3,2015,1,4,13,9,5,10,6,-3,93,...,1012,1004,16,9,1,47,10,74,7.62,6
4,2015,1,5,9,2,-6,-3,-12,-19,49,...,1020,1009,16,16,16,39,17,61,0,5


In [36]:
for i in list(range(22)):
    weather_2015.iloc[:,i]=pd.to_numeric(weather_2015.iloc[:,i], errors='coerce')

In [37]:
weather_2015 = weather_2015.drop(labels = 'Day', axis = 1)

In [38]:
weather_2015_group=weather_2015.groupby(['Year','Month']).transform('median')

In [39]:
mt=weather_2015.iloc[:,1]
yr=weather_2015.iloc[:,0]
weather_2015_group = weather_2015_group.assign(Month=mt.values)
weather_2015_group = weather_2015_group.assign(Year=yr.values)
weather_2015_group.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,Year
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
1,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
2,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
3,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
4,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015


In [40]:
drop_df_1=weather_2015_group.drop_duplicates(subset=None, keep='first', inplace=False)
drop_df_1.shape

(12, 22)

In [41]:
drop_df_2=drop_df_1.append(drop_df_1)
drop_df_2.shape

(24, 22)

In [42]:
drop_df_3=drop_df_2.append(drop_df_1)
drop_df_3.shape

(36, 22)

In [43]:
final_test=drop_df_3.sort_values(['Year','Month'], ascending=True)
final_test.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,Year
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2015
31,0.0,-4.0,-9.5,-6.5,-12.0,-19.0,68.0,54.5,34.0,1021.5,...,16.0,15.0,9.5,27.0,11.0,42.0,0.0,5,2,2015
31,0.0,-4.0,-9.5,-6.5,-12.0,-19.0,68.0,54.5,34.0,1021.5,...,16.0,15.0,9.5,27.0,11.0,42.0,0.0,5,2,2015


In [44]:
cat1 = test_feature.iloc[:,2]
final_test= final_test.assign(ProductCategory = cat1.values)
final_test= final_test.drop(labels = "Year", axis =1)

In [45]:
final_test.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent,Month,ProductCategory
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,3
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,1
0,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,16.0,16.0,14.0,26.0,10.0,40.0,0.0,5,1,2
31,0.0,-4.0,-9.5,-6.5,-12.0,-19.0,68.0,54.5,34.0,1021.5,...,16.0,15.0,9.5,27.0,11.0,42.0,0.0,5,2,3
31,0.0,-4.0,-9.5,-6.5,-12.0,-19.0,68.0,54.5,34.0,1021.5,...,16.0,15.0,9.5,27.0,11.0,42.0,0.0,5,2,1


In [46]:
standardizedData_train = StandardScaler().fit_transform(final)
X_Train = standardizedData_train
Y_Train = train.Sales

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [47]:
standardizedData_test = StandardScaler().fit_transform(final_test)
X_Test = standardizedData_test

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [48]:
#X_Train.shape, Y_Train.shape, X_Test.shape

In [49]:
#DTR = DecisionTreeRegressor(random_state = 0)
#DTR.fit (X_Train, Y_Train)
#Y_pred = DTR.predict(X_Test)

In [50]:
#sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
#sub['Sales'] = Y_pred
#sub.to_csv("sample_submission_day2_weather_5.csv", index = False)
#sub.head()

# macro_economic

In [51]:
micro=pd.read_excel("macro_economic.xlsx")

In [52]:
#micro.isnull().sum()

In [53]:
micro1= micro.apply(lambda x: x["Year-Month"].split("-"), axis=1)
#micro1

In [54]:
#appending the year to a
a=[]
for i in range(96):    
    a.append(micro1[i][0])

In [55]:
#appending the month to b
b=[]
for i in range(96):    
    b.append(micro1[i][1])

In [56]:
#assigning the year feature to micro dataframe
micro["Year"]=a

In [57]:
micro["Month"]=b

In [58]:
#droping the year-month col
micro= micro.drop(labels = "Year-Month", axis = 1)

In [59]:
micro.Month.replace([' Jan', ' Feb', ' Mar', ' Apr', ' May', ' Jun', ' Jul', ' Aug', ' Sep', ' Oct', ' Nov', ' Dec',]
                   ,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

In [60]:
for i in list (range (19)):
    micro.iloc[:,i]=pd.to_numeric(micro.iloc[:,i], errors='coerce')

In [61]:
micro= micro.drop(labels = "PartyInPower", axis = 1)

In [62]:
micro= micro.drop(labels = "AdvertisingExpenses (in Thousand Dollars)", axis = 1)

In [63]:
train_micro=micro.iloc[:72]
train_micro.head()

,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,Commercial Bank Interest Rate on Credit Card Plans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports,Year,Month
0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550,2009,1
1,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100,2009,2
2,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650,2009,3
3,14326.815525,14351.601731,235.582,9.0,12.97,11.05,22.13,56.78,10.25,9.296,7.559,787,12.400,3.62,12.225,2009,4
4,14345.904809,14368.123959,235.975,9.4,13.32,11.25,22.04,61.95,9.11,9.297,7.400,803,12.384,3.52,12.300,2009,5


In [64]:
train_micro.shape

(72, 17)

In [65]:
train_micro1=train_micro.append(train_micro)
train_micro1.shape

(144, 17)

In [66]:
final_micro_train=train_micro1.append(train_micro)
final_micro_train = final_micro_train.sort_values(['Year','Month'], ascending=True)

In [67]:
final_micro_train = final_micro_train.drop(labels = "Year", axis =1)
final_micro_train.head(20)

,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,Commercial Bank Interest Rate on Credit Card Plans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports,Month
0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550,1
0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550,1
0,14421.752895,14407.053343,233.402,7.8,12.03,11.44,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550,1
1,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100,2
1,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100,2
1,14389.200466,14366.176571,234.663,8.3,12.97,11.05,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100,2
2,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650,3
2,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650,3
2,14340.701639,14351.786822,235.067,8.7,12.97,11.05,22.22,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650,3
3,14326.815525,14351.601731,235.582,9.0,12.97,11.05,22.13,56.78,10.25,9.296,7.559,787,12.400,3.62,12.225,4


In [68]:
test_micro=micro.iloc[72:84]
test_micro.head()

,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,Commercial Bank Interest Rate on Credit Card Plans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports,Year,Month
72,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,2015,1
73,17931.046190,16400.926737,259.240,5.5,11.98,9.85,25.05,69.84,3.70,10.845,9.518,781,15.496,3.625,10.20,2015,2
74,17900.981857,16329.523792,259.647,5.4,11.98,9.85,25.04,69.35,-0.70,10.845,9.518,781,15.496,3.625,10.25,2015,3
75,18015.748120,16427.136671,259.959,5.4,11.98,9.85,24.94,71.70,3.39,10.845,9.518,794,15.735,3.625,10.25,2015,4
76,18075.281412,16434.482376,261.066,5.5,12.04,9.69,24.88,72.86,1.62,10.845,9.157,826,15.753,3.625,10.30,2015,5


In [69]:
test_micro.shape

(12, 17)

In [70]:
test_micro1=test_micro.append(test_micro)
test_micro1.shape

(24, 17)

In [71]:
final_micro_test = test_micro1.append(test_micro)
final_micro_test = final_micro_test.sort_values(['Year','Month'], ascending=True)

In [72]:
final_micro_test = final_micro_test.drop(labels = "Year", axis =1)
final_micro_test.head()

,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,unemployment rate,Commercial Bank Interest Rate on Credit Card Plans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports,Month
72,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,1
72,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,1
72,17792.116953,16319.786314,258.376,5.7,11.99,10.47,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,1
73,17931.046190,16400.926737,259.240,5.5,11.98,9.85,25.05,69.84,3.70,10.845,9.518,781,15.496,3.625,10.20,2
73,17931.046190,16400.926737,259.240,5.5,11.98,9.85,25.05,69.84,3.70,10.845,9.518,781,15.496,3.625,10.20,2


In [73]:
final.shape, final_test.shape, final_micro_train.shape, final_micro_test.shape

((216, 22), (36, 22), (216, 16), (36, 16))

In [74]:
names = [final, final_test, final_micro_train, final_micro_test]
for i in names:
    i.index = i['Month']
    print(i.shape)

(216, 22)
(36, 22)
(216, 16)
(36, 16)


In [75]:
#final.head()

In [76]:
#final_test.head()

In [77]:
#final_micro_train.head()

In [78]:
#final_micro_test.head()

In [79]:
train_final=pd.concat([final,final_micro_train],axis=1)
train_final.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports,Month
Month,,,,,,,,,,,,,,,,,,,,,
1,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.55,1
1,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.55,1
1,1.0,-2.0,-5.0,-5.0,-10.0,-13.0,69.0,54.0,39.0,1022.0,...,22.05,57.70,4.02,9.296,7.559,799,12.589,4.17,11.55,1
2,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.10,2
2,7.0,3.0,-2.0,-2.5,-6.5,-12.0,65.0,50.0,37.0,1023.0,...,22.22,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.10,2


In [80]:
train_final.shape

(216, 38)

In [81]:
test_final=pd.concat([final_test,final_micro_test],axis=1)
test_final.head()

,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,Humidity (%) avg,Humidity (%) low,Sea Level Press. (hPa) high,...,Earnings or wages in dollars per hour,Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports,Month
Month,,,,,,,,,,,,,,,,,,,,,
1,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,1
1,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,1
1,3.0,-1.0,-6.0,-7.0,-11.0,-16.0,69.0,52.0,35.0,1025.0,...,24.88,67.35,-1.39,10.845,9.518,760,15.496,3.775,9.45,1
2,0.0,-4.0,-9.5,-6.5,-12.0,-19.0,68.0,54.5,34.0,1021.5,...,25.05,69.84,3.70,10.845,9.518,781,15.496,3.625,10.20,2
2,0.0,-4.0,-9.5,-6.5,-12.0,-19.0,68.0,54.5,34.0,1021.5,...,25.05,69.84,3.70,10.845,9.518,781,15.496,3.625,10.20,2


In [82]:
test_final.shape

(36, 38)

In [83]:
standardizedData_train1 = StandardScaler().fit_transform(train_final)
X_Train1 = standardizedData_train1
Y_Train1 = train.Sales

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [84]:
standardizedData_test1 = StandardScaler().fit_transform(test_final)
X_Test1 = standardizedData_test1

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [85]:
X_Train1.shape, Y_Train1.shape, X_Test1.shape

((216, 38), (216,), (36, 38))

In [86]:
DTR1 = DecisionTreeRegressor(random_state = 0)
DTR1.fit (X_Train1, Y_Train1)
Y_pred_DTR = DTR1.predict(X_Test1)

In [87]:
sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
sub['Sales'] = Y_pred_DTR
sub.to_csv("sample_submission_day3_usingDTR_1.csv", index = False)
sub.head(5)

,S.No.,Sales
0,1,1755.0
1,2,524.0
2,3,921.0
3,4,3102.0
4,5,524.0


In [88]:
KNN1 = KNeighborsRegressor()
KNN1.fit(X_Train1, Y_Train1)
Y_pred_KNN = KNN1.predict(X_Test1)

In [89]:
sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
sub['Sales'] = Y_pred_KNN
sub.to_csv("sample_submission_day3_usingKNN_1.csv", index = False)
sub.head(5)

,S.No.,Sales
0,1,1155.0
1,2,908.8
2,3,908.8
3,4,1904.6
4,5,991.2


In [90]:
#LOGR1 = LogisticRegression()
#LOGR1.fit(X_Train1, Y_Train1)
#Y_pred_LOGR = LOGR1.predict(X_Test1)

In [91]:
#sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
#sub['Sales'] = Y_pred_LOGR
#sub.to_csv("sample_submission_day3_usingLOGR_1.csv", index = False)
#sub.head(5)

In [92]:
from sklearn.gaussian_process import GaussianProcessRegressor

HR1 = HuberRegressor()
HR1.fit(X_Train1, Y_Train1)
Y_pred_HR = HR1.predict(X_Test1)

In [93]:
sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
sub['Sales'] = Y_pred_HR
sub.to_csv("sample_submission_day3_usingHR_1.csv", index = False)
sub.head(5)

,S.No.,Sales
0,1,1484.421312
1,2,-989.759645
2,3,247.330834
3,4,2190.389254
4,5,-283.791703


In [94]:
#LINR1 = LinearRegression()
#LINR1.fit(Y_Train1, X_Train1)
#Y_pred_LINR = LINR1.predict(X_Test1)
LINR1 = sm.OLS(Y_Train1, X_Train1).fit()
Y_pred_LINR = LINR1.predict(X_Test1)

In [95]:
sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
sub['Sales'] = Y_pred_LINR
sub.to_csv("sample_submission_day3_usingLINR_1.csv", index = False)
sub.head(6)

,S.No.,Sales
0,1,-400.189069
1,2,-3024.439069
2,3,-1712.314069
3,4,524.297745
4,5,-2099.952255
5,6,-787.827255


In [96]:
GPR1 = GaussianProcessRegressor()
GPR1.fit(X_Train1, Y_Train1)
Y_pred_GPR = GPR1.predict(X_Test1)

In [97]:
sub = pd.DataFrame(pd.read_csv("sample_submission.csv")['S.No.'])
sub['Sales'] = Y_pred_GPR
sub.to_csv("sample_submission_day3_usingGPR_1.csv", index = False)
sub.head(5)

,S.No.,Sales
0,1,0.000183
1,2,0.000052
2,3,0.000091
3,4,0.013641
4,5,0.003044


# EDA

In [98]:
df_final = train_final
sales = train['Sales']
df_final['Sales']= sales

In [99]:
df_final = df_final.drop(labels = "Visibility (km) high", axis =1)

KeyError: "['Visibility (km) high'] not found in axis"

In [ ]:
df_final.isnull().sum()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(df_final.corr(), annot=True, linewidths=.5, center=0, cbar=False, cmap="YlGnBu")
plt.title('Heatemap of Correlation Matrix')
plt.show()

In [ ]:
df_final.corr()